---
title: "Homework 2: Poisson Regression Examples"
author: "Alex Wissman"
date: today
callout-appearance: minimal # this hides the blue "i" icon on .callout-notes
---


## Blueprinty Case Study

### Introduction

Blueprinty is a small firm that makes software for developing blueprints specifically for submitting patent applications to the US patent office. Their marketing team would like to make the claim that patent applicants using Blueprinty's software are more successful in getting their patent applications approved. Ideal data to study such an effect might include the success rate of patent applications before using Blueprinty's software and after using it. Unfortunately, such data is not available. 

However, Blueprinty has collected data on 1,500 mature (non-startup) engineering firms. The data include each firm's number of patents awarded over the last 5 years, regional location, age since incorporation, and whether or not the firm uses Blueprinty's software. The marketing team would like to use this data to make the claim that firms using Blueprinty's software are more successful in getting their patent applications approved.


### Data

Blueprinty customers are not selected at random. It may be important to account for systematic differences in the age and regional location of customers vs non-customers.

In [ ]:
import pandas as pd
import scipy as sp
import numpy as np
import statsmodels.api as sm
import pyrsm as rsm
import matplotlib.pyplot as plt

# Load and read the files
airbnb_data = pd.read_csv('/home/jovyan/mysite/airbnb.csv')
blueprinty_data = pd.read_csv('/home/jovyan/mysite/blueprinty.csv')

# Display the first few rows of the dataset
print(airbnb_data.head())

In [ ]:
# Group the data by customer status and calculate the mean number of patents
means = blueprinty_data.groupby('iscustomer')['patents'].mean()
print(means)

In [ ]:
# Plot histograms for each customer status
statuses = blueprinty_data['iscustomer'].unique()
for status in statuses:
    subset = blueprinty_data[blueprinty_data['iscustomer'] == status]
    plt.hist(subset['patents'], bins=10, alpha=0.5, label=f'Status: {status}')
    if status == 0:
        plt.axvline(means[status], color='blue', linestyle='dashed', linewidth=1, label=f'Mean for {status}')
    else:
        plt.axvline(means[status], color='green', linestyle='dotted', linewidth=1, label=f'Mean for {status}')

plt.xlabel('Number of Patents')
plt.ylabel('Frequency')
plt.title('Histograms of Number of Patents by Customer Status')
plt.legend()
plt.show()

The histogram reveals that companies who are customers of the software service tend to have a slightly higher mean number of patents (4.13) compared to companies who are not customers (3.47). 
This suggests that being a customer of the software service may correlate with a higher number of patents, though further analysis would be needed to establish causation.

Blueprinty customers are not selected at random. It may be important to account for systematic differences in the age and regional location of customers vs non-customers.

In [ ]:
# Group by customer status and analyze region distribution
region_distribution = blueprinty_data.groupby(['iscustomer', 'region']).size().unstack(fill_value=0)

# Group by customer status and calculate mean age
age_mean = blueprinty_data.groupby('iscustomer')['age'].mean()

print("Region Distribution by Customer Status:")
print(region_distribution)

print("\nMean Age by Customer Status:")
print(age_mean)

Observations:
1. The Northeast region has the highest number of customers (328) compared to other regions.
2. The Midwest region has the lowest number of customers (37).
3. Non-customers are more evenly distributed across regions, with the Southwest region having the highest count (245) and the Northwest region having the lowest count (158).
4. Customers are concentrated more in the Northeast region, while non-customers are more prevalent in the Southwest region.
5. The South region has a relatively low number of both customers (35) and non-customers (156).

### Estimation of Simple Poisson Model

Since our outcome variable of interest can only be small integer values per a set unit of time, we can use a Poisson density to model the number of patents awarded to each engineering firm over the last 5 years. We start by estimating a simple Poisson model via Maximum Likelihood.

In [ ]:
from sympy import symbols, factorial, exp, prod

# Define variables
Y, lam = symbols('Y lambda', positive=True, integer=True)

# Poisson probability mass function
poisson_pmf = (exp(-lam) * lam**Y) / factorial(Y)

# Likelihood for a dataset of observations
observations = symbols('Y1 Y2 Y3 Y4', positive=True, integer=True)  # Example observations
likelihood = prod((exp(-lam) * lam**obs) / factorial(obs) for obs in observations)

print("Poisson PMF:", poisson_pmf)
print("Likelihood for observations:", likelihood)

In [ ]:
from sympy import log, lambdify

# Define the log-likelihood function
def poisson_loglikelihood(lam, observations):
    log_likelihood = sum(log((exp(-lam) * lam**obs) / factorial(obs)) for obs in observations)
    return log_likelihood

# Example usage
log_likelihood_expr = poisson_loglikelihood(lam, observations)
print("Log-Likelihood Expression:", log_likelihood_expr)

# Optionally, convert to a numerical function
log_likelihood_func = lambdify((lam, observations), log_likelihood_expr)

In [ ]:
# Extract the observed number of patents
observed_patents = blueprinty_data['patents'].values

# Define a range of lambda values
lambda_values = np.linspace(0.1, 10, 100)

# Compute the log-likelihood for each lambda
log_likelihoods = [poisson_loglikelihood(lam_val, observed_patents) for lam_val in lambda_values]

# Plot the log-likelihood
plt.figure(figsize=(10, 6))
plt.plot(lambda_values, log_likelihoods, label='Log-Likelihood', color='blue')
plt.xlabel('Lambda')
plt.ylabel('Log-Likelihood')
plt.title('Log-Likelihood vs Lambda')
plt.legend()
plt.grid()
plt.show()

In [ ]:
from scipy.optimize import minimize

# Define a numerical version of the log-likelihood function
def numerical_log_likelihood(lam_val):
    return -sum(np.log((np.exp(-lam_val) * lam_val**obs) / np.math.factorial(obs)) for obs in observed_patents)

# Initial guess for lambda
initial_guess = 1.0

# Perform the optimization
result = minimize(numerical_log_likelihood, initial_guess, bounds=[(0.1, None)])

# Extract the MLE for lambda
mle_lambda = result.x[0]
print("MLE for lambda:", mle_lambda)

### Estimation of Poisson Regression Model

Next, we extend our simple Poisson model to a Poisson Regression Model such that $Y_i = \text{Poisson}(\lambda_i)$ where $\lambda_i = \exp(X_i'\beta)$. The interpretation is that the success rate of patent awards is not constant across all firms ($\lambda$) but rather is a function of firm characteristics $X_i$. Specifically, we will use the covariates age, age squared, region, and whether the firm is a customer of Blueprinty.

In [ ]:
from sympy import Matrix, exp, symbols

# Define the Poisson regression likelihood function
def poisson_regression_likelihood(beta, Y, X):
    # Convert beta and X to symbolic matrices
    beta = Matrix(beta)
    X = Matrix(X)
    
    # Compute lambda_i = exp(X_i' * beta) for each observation
    lambdas = (X * beta).applyfunc(exp)
    
    # Compute the likelihood
    likelihood = prod((lambdas[i]**Y[i] * exp(-lambdas[i])) / factorial(Y[i]) for i in range(len(Y)))
    return likelihood

# Example usage
beta = symbols('beta0 beta1', real=True)  # Example beta vector
X = [[1, 2], [1, 3], [1, 4]]  # Example covariate matrix
Y = [2, 3, 4]  # Example observations

likelihood = poisson_regression_likelihood(beta, Y, X)
print("Poisson Regression Likelihood:", likelihood)

Identify the MLE vector and the Hessian of the Poisson model with covariates.

In [ ]:
from scipy.optimize import minimize
from sympy import exp

# Prepare the covariate matrix X
blueprinty_data['age_squared'] = blueprinty_data['age'] ** 2
region_dummies = pd.get_dummies(blueprinty_data['region'], drop_first=True)
X = pd.concat([pd.Series(1, index=blueprinty_data.index, name='Intercept'),
               blueprinty_data['age'], blueprinty_data['age_squared'],
               region_dummies, blueprinty_data['iscustomer']], axis=1).astype(float).values

# Response variable Y
Y = blueprinty_data['patents'].values

# Define the Poisson regression log-likelihood function
def poisson_regression_loglikelihood(beta, Y, X):
    lambdas = np.exp(np.dot(X, beta))
    log_likelihood = np.sum(Y * np.log(lambdas) - lambdas - np.log(np.array([np.math.factorial(y) for y in Y])))
    return -log_likelihood  # Negative for minimization

# Initial guess for beta
initial_beta = np.zeros(X.shape[1])

# Perform the optimization
result = minimize(poisson_regression_loglikelihood, initial_beta, args=(Y, X), method='BFGS')

# Extract the MLE for beta and the Hessian
mle_beta = result.x
hessian_inv = result.hess_inv

# Calculate standard errors from the Hessian
standard_errors = np.sqrt(np.diag(hessian_inv))

# Create a table of coefficients and standard errors
coefficients_table = pd.DataFrame({
    'Coefficient': mle_beta,
    'Standard Error': standard_errors
}, index=['Intercept', 'Age', 'Age Squared'] + list(region_dummies.columns) + ['Is Customer'])

print(coefficients_table)

In [ ]:
import statsmodels.api as sm

# Fit a Poisson regression model using GLM
poisson_model = sm.GLM(Y, X, family=sm.families.Poisson())
poisson_results = poisson_model.fit()

# Print the summary of the model
print(poisson_results.summary())

The results of the generalized linear regression model can be interpreted as follows:

1. **Intercept (`const`)**: The coefficient for the intercept is -0.5089, which represents the baseline log-expected value of the dependent variable (`y`) when all predictors are zero. This value is statistically significant (p-value = 0.005).

2. **Predictor `x1`**: The coefficient for `x1` is 0.1486, indicating that for a one-unit increase in `x1`, the expected value of `y` increases by approximately 14.86% (since the link function is log, the effect is multiplicative). This is highly significant (p-value < 0.001).

3. **Predictor `x2`**: The coefficient for `x2` is -0.0030, suggesting that a one-unit increase in `x2` decreases the expected value of `y` by approximately 0.3%. This is also highly significant (p-value < 0.001).

4. **Predictors `x3`, `x4`, `x5`, and `x6`**: These predictors have coefficients close to zero and high p-values (greater than 0.05), indicating that they are not statistically significant in explaining the variation in `y`.

5. **Predictor `x7`**: The coefficient for `x7` is 0.2076, meaning that a one-unit increase in `x7` increases the expected value of `y` by approximately 20.76%. This is statistically significant (p-value < 0.001).

6. **Model Fit**:
    - **Log-Likelihood**: The log-likelihood value is -3258.1, which reflects the fit of the model to the data.
    - **Deviance**: The deviance is 2143.3, which measures the goodness of fit. Lower values indicate a better fit.
    - **Pseudo R-squared (CS)**: The pseudo R-squared value is 0.1360, suggesting that the model explains about 13.6% of the variability in the dependent variable.

7. **Significance**: Predictors `x1`, `x2`, and `x7` are statistically significant, while the others are not. This suggests that these three variables are the most important in explaining the variation in `y`.

In summary, the model identifies `x1`, `x2`, and `x7` as significant predictors of the dependent variable, while the other predictors do not contribute significantly. The overall fit of the model is moderate, as indicated by the pseudo R-squared value.


## AirBnB Case Study

### Introduction

AirBnB is a popular platform for booking short-term rentals. In March 2017, students Annika Awad, Evan Lebo, and Anna Linden scraped of 40,000 Airbnb listings from New York City.  The data include the following variables:

:::: {.callout-note collapse="true"}
### Variable Definitions

    - `id` = unique ID number for each unit
    - `last_scraped` = date when information scraped
    - `host_since` = date when host first listed the unit on Airbnb
    - `days` = `last_scraped` - `host_since` = number of days the unit has been listed
    - `room_type` = Entire home/apt., Private room, or Shared room
    - `bathrooms` = number of bathrooms
    - `bedrooms` = number of bedrooms
    - `price` = price per night (dollars)
    - `number_of_reviews` = number of reviews for the unit on Airbnb
    - `review_scores_cleanliness` = a cleanliness score from reviews (1-10)
    - `review_scores_location` = a "quality of location" score from reviews (1-10)
    - `review_scores_value` = a "quality of value" score from reviews (1-10)
    - `instant_bookable` = "t" if instantly bookable, "f" if not

::::


In [ ]:
import matplotlib.pyplot as plt

# Drop observations with missing values in relevant columns
relevant_columns = ['number_of_reviews', 'price', 'room_type', 'bathrooms', 'bedrooms']
cleaned_airbnb_data = airbnb_data.dropna(subset=relevant_columns)

# Summary statistics for the number of reviews
print("Summary statistics for the number of reviews:")
print(cleaned_airbnb_data['number_of_reviews'].describe())

# Distribution of the number of reviews
plt.figure(figsize=(10, 6))
plt.hist(cleaned_airbnb_data['number_of_reviews'], bins=50, color='skyblue', edgecolor='black')
plt.xlabel('Number of Reviews')
plt.ylabel('Frequency')
plt.title('Distribution of Number of Reviews')
plt.grid(axis='y')
plt.show()

# Relationship between price and number of reviews
plt.figure(figsize=(10, 6))
plt.scatter(cleaned_airbnb_data['price'], cleaned_airbnb_data['number_of_reviews'], alpha=0.5, color='purple')
plt.xlabel('Price')
plt.ylabel('Number of Reviews')
plt.title('Price vs Number of Reviews')
plt.grid()
plt.show()

# Average number of reviews by room type
avg_reviews_by_room_type = cleaned_airbnb_data.groupby('room_type')['number_of_reviews'].mean()
print("\nAverage number of reviews by room type:")
print(avg_reviews_by_room_type)

# Bar plot for average number of reviews by room type
avg_reviews_by_room_type.plot(kind='bar', color='orange', edgecolor='black', figsize=(8, 5))
plt.xlabel('Room Type')
plt.ylabel('Average Number of Reviews')
plt.title('Average Number of Reviews by Room Type')
plt.grid(axis='y')
plt.show()

In [ ]:
# Prepare the covariate matrix X for the Airbnb data
# Include relevant variables such as price, room_type (encoded as dummies), bathrooms, and bedrooms
room_type_dummies = pd.get_dummies(cleaned_airbnb_data['room_type'], drop_first=True)
X_airbnb = pd.concat([cleaned_airbnb_data[['price', 'bathrooms', 'bedrooms']], room_type_dummies], axis=1)

# Convert boolean columns to integers
X_airbnb = X_airbnb.astype(float)

# Add a constant term for the intercept
X_airbnb = sm.add_constant(X_airbnb)

# Response variable Y (number of reviews)
Y_airbnb = cleaned_airbnb_data['number_of_reviews']

# Fit a Poisson regression model
poisson_model_airbnb = sm.GLM(Y_airbnb, X_airbnb, family=sm.families.Poisson())
poisson_results_airbnb = poisson_model_airbnb.fit()

# Print the summary of the model
print(poisson_results_airbnb.summary())

The coefficients from the Poisson regression model can be interpreted as follows:

1. **Intercept (`const`)**: The baseline log-expected number of reviews is approximately 2.9065 when all other variables are zero. This corresponds to an expected number of reviews of \( e^{2.9065} \approx 18.3 \).

2. **Price (`price`)**: For every one-unit increase in price, the expected number of reviews decreases by approximately \( e^{-0.0005} - 1 \approx -0.05\% \). This indicates a very small negative relationship between price and the number of reviews.

3. **Bathrooms (`bathrooms`)**: For every additional bathroom, the expected number of reviews decreases by approximately \( e^{-0.1052} - 1 \approx -10\% \). This suggests that listings with more bathrooms tend to have fewer reviews.

4. **Bedrooms (`bedrooms`)**: For every additional bedroom, the expected number of reviews increases by approximately \( e^{0.1042} - 1 \approx 10.97\% \). This indicates that listings with more bedrooms tend to have more reviews.

5. **Private Room (`Private room`)**: Compared to the baseline category (Entire home/apt), private rooms have an expected number of reviews that is approximately \( e^{-0.1398} - 1 \approx -13\% \) lower.

6. **Shared Room (`Shared room`)**: Compared to the baseline category (Entire home/apt), shared rooms have an expected number of reviews that is approximately \( e^{-0.3895} - 1 \approx -32.2\% \) lower.

### Summary:
- Listings with higher prices and more bathrooms tend to have fewer reviews.
- Listings with more bedrooms tend to have more reviews.
- Private and shared rooms receive fewer reviews compared to entire homes/apartments.